In [1]:
import pandas as pd
import numpy as np
# To show all the columns
pd.pandas.set_option('display.max_column',None)

messages = pd.read_csv(r'C:\Users\mohan\Desktop\Kaggle_competition\NLP\Spam Classifier\smsspamcollection\SMSSpamCollection',sep='\t',names=['label','message'])

messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# Data Cleaning

In [2]:
import re
import nltk


from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [3]:
ps = PorterStemmer()
wordnet = WordNetLemmatizer()
corpus=[]

for i in range (0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['message'][i]) # Substitute all numeric values and symbols with space ' '
    review=review.lower().split()
    
    # Eliminating the stop words and stemming others
    #review =[ps.stem(word) for word in review if not word in stopwords.words('English')]
    review =[wordnet.lemmatize(word) for word in review if not word in stopwords.words('English')]
    review= ' '.join(review)
    corpus.append(review)

# Creating Bag of words model

In [ ]:
# TFIDF

#from sklearn.feature_extraction.text import TfidfVectorizer
#cv = TfidfVectorizer(max_features=1500)
#X =cv.fit_transform(corpus).toarray()
#np.shape(X)

In [4]:
# Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X =cv.fit_transform(corpus).toarray()
np.shape(X)

(5572, 2500)

In [5]:
y= pd.get_dummies(messages['label'],drop_first=True)
y.shape


(5572, 1)

# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


# Training model using Naive bayes classifier


In [7]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, np.ravel(y_train))

# Evaluating the model

In [8]:
y_pred = spam_detect_model.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[946   9]
 [ 10 150]]
0.9829596412556054


# Classification of new SMS

In [9]:
def Predict_spam():
    lab2spam={1:'Spam',0:'Not Spam'}
    
    msg = input('Enter the SMS ')
    msg=re.sub('[^a-zA-Z]',' ',msg) # Substitute all numeric values and symbols with space ' '
    msg=msg.lower().split()
    msg =[wordnet.lemmatize(word) for word in msg if not word in stopwords.words('English')]
    msg= ' '.join(msg)
    msg =cv.transform([msg]).toarray()
    pred = spam_detect_model.predict(msg)
    print("\n This message is a "+str(lab2spam[pred[0]]))
    return 

In [10]:
Predict_spam()

Enter the SMS Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

 This message is a Spam
